In [1]:
import time
import pickle
import scipy.sparse

import numpy as np

from helpers.helpers import *

In [2]:
# Set of duplicate users
duplicate_users = dict_occurent_users()

# -------------------------------------------------------------------------------------------

### Number of comments per channel

Compute the number of comments per channel and store it into the YouTube project

In [3]:
vid_to_channels = pd.read_pickle("/dlabdata1/youtube_large/id_to_channel_mapping.pickle")

In [10]:
# Select only english channels
channels_id = pd.read_csv("/dlabdata1/youtube_large/df_channels_en.csv.gz")
channels_id = set(channels_id['channel'])

In [13]:
'''
Add a comment into the a dictionnary keeping track of the number of comments per channel
PARAMETERS:
    - dictionnary: dicionnary mapping a channel index with it's corresponding number of comments
    - corr_channel: the channel index in which we want to add a comment
'''
def add_comment(dictionnary, corr_channel):
    if corr_channel in dictionnary:
        dictionnary[corr_channel] += 1
    else:
        dictionnary[corr_channel] = 1

In [14]:
# Adjust chunk_size as necessary -- defaults to 16,384 if not specific
reader = Zreader("/dlabdata1/youtube_large/youtube_comments.ndjson.zst", chunk_size=160384)

# parameters
idx = 1
comments_per_channel = {}
user = ''
begin_time = time.time()


# Read each line from the reader
for line in reader.readlines():
    line_split = line.replace('"', '').split(',')
    if len(line_split) == 9:
        author_id = line_split[0]
        if vid_to_channels.get(line_split[2]) in channels_id:
            corr_channel = vid_to_channels[line_split[2]]
            if author_id == user:
                if author_id in duplicate_users:
                    if duplicate_users[author_id] <= 1:
                        add_comment(comments_per_channel, corr_channel)
                else:
                    add_comment(comments_per_channel, corr_channel)
            else:
                if author_id in duplicate_users:
                    duplicate_users[author_id] += 1
                    if duplicate_users[author_id] <= 1:
                        add_comment(comments_per_channel, corr_channel)
                else:
                    add_comment(comments_per_channel, corr_channel)
            
            user = author_id
    idx += 1
    if idx % 1000000000 == 0:
        print(begin_time-time.time())
        begin_time = time.time()
    

-6333.590427398682
-6144.500817298889
-6062.126186847687
-5905.280533075333
-6172.949314117432
-6818.088992357254
-6077.464460372925
-5907.094409942627
-5780.842119693756
-5753.591685295105


In [15]:
# Store the comments_per_channel dictionnary into the YouTube project
with open("/dlabdata1/youtube_large/jouven/comments_per_channel.pkl",'wb') as f:
     pickle.dump(comments_per_channel, f)
f.close()

In [ ]:
# Adjust chunk_size as necessary -- defaults to 16,384 if not specific
reader = Zreader("/dlabdata1/youtube_large/youtube_comments.ndjson.zst", chunk_size=160384)

# parameters
idx = 1
comments_per_channels = {}
users_per_channel = {}
user = ''
begin_time = time.time()


# Read each line from the reader
for line in reader.readlines():
    line_split = line.replace('"', '').split(',')
    if len(line_split) == 9:
        author_id = line_split[0]
        if vid_to_channels.get(line_split[2]) in channels_id:
            corr_channel = vid_to_channels[line_split[2]]
            if author_id == user:
                if author_id in occurent_users:
                    if occurent_users[author_id] <= 1:
                        add_comment(comments_per_channels, corr_channel)
                else:
                    add_comment(comments_per_channels, corr_channel)
            else:
                if author_id in occurent_users:
                    occurent_users[author_id] += 1
                    if occurent_users[author_id] <= 1:
                        add_comment(comments_per_channels, corr_channel)
                else:
                    add_comment(comments_per_channels, corr_channel)
            
            user = author_id
    idx += 1
    if idx % 1000000000 == 0:
        print(begin_time-time.time())
        begin_time = time.time()
    

# -------------------------------------------------------------------------------------------

### Video channel mapping filtered

From the original dictionnary having all the video-channel relationship, we only select data corresponding to the selected channels.

In [4]:
vid_to_channels = pd.read_pickle("/dlabdata1/youtube_large/id_to_channel_mapping.pickle")

In [6]:
# Set of channels selected
dict_channel_ind, dict_ind_channel, channels_id = filtered_channels_index_id_mapping()

In [7]:
vid_to_channels_filtered = {}
for vid, channel in vid_to_channels.items():
    if not dict_channel_ind.get(channel) == None:
        if channel in channels_id:
            vid_to_channels_filtered[vid] = channel

In [8]:
len(vid_to_channels_filtered)

48254261

In [9]:
# Store the  video_id to the channel index filtered mapping
with open("/dlabdata1/youtube_large/jouven/video_to_channel_mapping_filtered.pkl",'wb') as f:
     pickle.dump(vid_to_channels_filtered, f)
f.close()

# -------------------------------------------------------------------------------------------

### Compute the number of users after filtering the set of channels taken

In [3]:
# Video to channel mapping
vid_to_channels = video_id_to_channel_id()

In [7]:
vid_to_channels = pd.read_pickle("/dlabdata1/youtube_large/jouven/video_to_channel_mapping_filtered.pkl")

In [10]:
# Set of channels with more than 10k comments
dict_channel_ind, dict_ind_channel, channels_id = filtered_channels_index_id_mapping()

In [11]:
# Adjust chunk_size as necessary -- defaults to 16,384 if not specific
reader = Zreader("/dlabdata1/youtube_large/youtube_comments.ndjson.zst", chunk_size=160384)

# parameters
idx = 1
comments_per_channels = {}
users_per_channel = {}
user = ''
begin_time = time.time()
nb_user = -1


# Read each line from the reader
for line in reader.readlines():
    line_split = line.replace('"', '').split(',')
    if len(line_split) == 9:
        author_id = line_split[0]
        if vid_to_channels.get(line_split[2]) in channels_id:
            corr_channel = vid_to_channels[line_split[2]]
            if author_id != user:
                if author_id in occurent_users:
                    occurent_users[author_id] += 1
                    if occurent_users[author_id] <= 1:
                        nb_user += 1
                else:
                    nb_user += 1
            
            user = author_id
    idx += 1
    if idx % 1000000000 == 0:
        print('Line number' + str(idx) + str(time.time()-begin_time))
        begin_time = time.time()
    

-5458.276359081268
-5251.27592587471
-5658.101206541061
-5454.000999212265
-5539.64678311348
-5518.447799921036
-5618.892131090164
-5525.270644187927
-5547.37446808815
-5535.287395238876


In [17]:
with open("/dlabdata1/youtube_large/jouven/nb_users.pkl",'rb') as f:
     print(pickle.load(f))
f.close()

[406925230]


In [13]:
nb_user

406925230

### Number of users that commented a channel

In [ ]:
bipartite_graph_sparse_matrix = scipy.sparse.load_npz('/dlabdata1/youtube_large/jouven/final_embedding_sparse_matrix/sparse_matrix_bipartite.npz')

In [ ]:
oli_dict_channel_ind, oli_dict_ind_channel, oli_channels_id = channels_filtered_oli()

In [ ]:
users_per_channel = {}
for i in range(bipartite_graph_sparse_matrix.shape[0]):
    users_per_channel[oli_dict_ind_channel[i]] = bipartite_graph_sparse_matrix[[i],:].count_nonzero()

In [ ]:
# Store the users_per_channel dictionnary into the YouTube project
with open("/dlabdata1/youtube_large/jouven/users_per_channel_id.pkl",'wb') as f:
     pickle.dump(users_per_channel, f)
f.close()